### Installs, Imports, and Pyro Setup 

In [1]:
# 1# Run first time
# ! module load anaconda3/2022.05-gcc/9.5.0 cuda/11.1.1-gcc/9.5.0 cudnn/8.0.5.39-11.1-gcc/9.5.0-cu11_1
# ! source activate updated_pytorch
# !pip install --up
# !pip install pyro-ppl
# !pip install torchvision
# !pip install --upgrade git+https://github.com/dhudsmith/clean-the-text
# !pip install umap-learn

In [2]:
from dataset import PairwiseData, DocumentPairData
from model import Encoder, Decoder, ProdLDA, custom_elbo
import os
import pickle as pkl
from typing import List

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# import torchvision.transforms as transforms
import pandas as pd


import pyro
from pyro import poutine
import pyro.distributions as dist
# import pyro.contrib.examples.util  # patches torchvision
from pyro.infer import SVI, Trace_ELBO, TraceMeanField_ELBO
from pyro.optim import Adam, ClippedAdam
import torch.nn.functional as F
from scipy.io import loadmat

import scipy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import wandb
import umap
import umap.plot
import sklearn.datasets

plt.rcParams["figure.figsize"] = (10,5)

/home/dsaad/.conda/envs/updated_pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pyro.distributions.enable_validation(True)

In [4]:
pyro.set_rng_seed(0)

# Setup

In [5]:
# Data
NUMBER_PAIRS = 100000


NUM_WORKERS = 0

# Dropout rate
NUM_TOPICS = 50
NUM_PROTOTYPES = 5
EMBED_DIM  = 64
HIDDEN_DIM = 128
DROPOUT_RATE = 0.2

# Training
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 75
TEST_FREQUENCY = 1
BATCH_SIZE = 128
VAL_BATCH_SIZE = 2000
OBS_SAMPLES = 10

# gamma = 0.25
# num_steps = (NUMBER_PAIRS // BATCH_SIZE) * NUM_EPOCHS
# lrd = np.exp(np.log(gamma)/num_steps)
adam_args = {"lr": 0.003, 'clip_norm':10.0, 'betas': (0.99, 0.999)}

latest_plotted_p = torch.zeros(NUM_PROTOTYPES, NUM_TOPICS)

In [6]:
torch.cuda.current_device()

0

# Data Prep

In [7]:
def get_data(pair_percentage=1.0):
    # pairwise data
    pwd = PairwiseData()
    train_pairs, val_pairs, test_pairs = [pwd.get_pairs_table(d) for d in [pwd.train, pwd.val, pwd.test]]

    # datasets
    data_train, data_val, data_test = [
        DocumentPairData(bows=pwd.bows, index_table=ix_table, prob=pair_percentage)
        for ix_table in [train_pairs, val_pairs, test_pairs]
    ]

    # dataloaders
    dl_train = DataLoader(data_train, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
    dl_val = DataLoader(data_val, batch_size=VAL_BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)
    dl_test = DataLoader(data_test, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)
    
    return pwd, dl_train, dl_val, dl_test

# Training and Evaluation Functions 

In [8]:
def train(svi, topic_model, train_loader, device, progress_interval = 100, print_debug = False, epoch = 0, latest_plotted_p = None, trans = None):
    topic_model.train()
    
    # initialize loss accumulator
    epoch_loss = 0.
    logits_epoch = []
    labels_epoch = []
    batches = 0
    
    
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for i, x in enumerate(train_loader):
        batches += 1
        # if on GPU put mini-batch into CUDA memory
        x_a = x['a'].to(device).squeeze()
        x_b = x['b'].to(device).squeeze()
        x_label = x['label'].to(device).type(torch.float32)
        x_observed = x['observed'].to(device).type(torch.bool)
        
        if print_debug and i % progress_interval == 0:
            print(f"Step {i}; avg. loss {epoch_loss/(i+1)}", end='\r')
            # p = pyro.param('p').detach().cpu()
            # phi = torch.softmax(p, axis=-1)
            # p_2d = trans.transform(p)
            # latest_plotted_p[:] = p
            # umap.plot.points(trans, labels=pwd.val.category,theme='fire')
            # for ix, marker in enumerate(['$1$', '$2$', '$3$', '$4$', '$5$', '$6$', '$7$']):
            #     plt.scatter(x=p_2d[ix,0], y=p_2d[ix,1], s=7**3, c='white', marker=marker)
            # plt.title(f"Frozen Enc/Dec : Epoch {epoch}, Step {i}")
            # plt.savefig(f'../data/figures_pretrained_unfrozen/epoch{epoch}_step{i}')
            # plt.close()
        
        # do ELBO gradient and accumulate loss
        epoch_loss += svi.step(x_a, x_b, x_label, x_observed)
        
        with torch.no_grad():
            x = svi.guide(x_a, x_b, x_label, x_observed)

        x_label = x_label.cpu()
        logits = x.cpu()
        logits_epoch += logits
        labels_epoch += x_label
        
    logits_binned = np.digitize(logits_epoch, [0.45], right=False)
    train_acc = accuracy_score(labels_epoch, logits_binned)
    train_auroc = roc_auc_score(labels_epoch, logits_epoch)
    
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    return total_epoch_loss_train, train_acc, train_auroc

In [9]:
def evaluate(svi, etm, test_loader, device): 
    etm.eval()
    
    # initialize loss accumulator
    test_loss = 0.0
    test_ce_loss = 0.0
    test_kl_loss = 0.0
    test_acc = 0.0
    test_auroc = 0.0
    batches = 0
    logits_epoch = []
    labels_epoch = []
    
    # compute the loss over the entire test set
    for x in test_loader:
        batches += 1
        # if on GPU put mini-batch into CUDA memory
        x_a = x['a'].to(device).squeeze()
        x_b = x['b'].to(device).squeeze()
        x_label = x['label'].to(device).type(torch.float32)
        x_observed = x['observed'].to(device).type(torch.bool)
            
        # compute ELBO estimate and accumulate loss
        test_loss += svi.evaluate_loss(x_a, x_b, x_label, x_observed)
        
        # generate reconstruction of batch documents and move to cuda if designated
        with torch.no_grad():
            recon_x_a = etm.reconstruct_doc(x_a)
            recon_x_b = etm.reconstruct_doc(x_b)
            logits = svi.guide(x_a, x_b, x_label, x_observed)

        x_label = x_label.cpu()
        logits = logits.cpu()
        logits_epoch += logits
        labels_epoch += x_label

        # calculate and sum cross entropy loss and kl divergence
        x_a=x_a.squeeze()
        x_b=x_b.squeeze()
        log_probs_a = torch.log(recon_x_a)
        log_probs_b = torch.log(recon_x_b)
        
        if log_probs_a.isnan().any() | log_probs_b.isnan().any() | (log_probs_a.abs() > 30).any() | (log_probs_b.abs() > 30).any():
            raise ValueError("nan or very large log probs")
            
        targets_a = x_a/(x_a.sum(axis=-1)[:,None])
        targets_b = x_b/(x_b.sum(axis=-1)[:,None])
        ce_loss_a = F.cross_entropy(log_probs_a, targets_a, reduction='sum')
        ce_loss_b = F.cross_entropy(log_probs_b, targets_b, reduction='sum')
        test_ce_loss += ce_loss_a + ce_loss_b
        test_kl_loss += etm.calc_kl_divergence(x_a.squeeze(1)) + etm.calc_kl_divergence(x_b.squeeze(1))

        
    logits_binned = np.digitize(logits_epoch, [0.45], right=False)
    test_acc = accuracy_score(labels_epoch, logits_binned)
    test_auroc = roc_auc_score(labels_epoch, logits_epoch)
    
    normalizer_test = len(test_loader.dataset)
    epoch_elbo = test_loss / normalizer_test
    epoch_ce_loss = test_ce_loss / normalizer_test
    epoch_kl_loss = test_kl_loss / normalizer_test
    
    return epoch_elbo, epoch_ce_loss.item(), epoch_kl_loss.item(), test_acc, test_auroc

## Building Model

In [10]:
def init_model(pretrained, frozen, pwd):
    try:
        if frozen and not pretrained:
            raise Exception
    except:
        print('ERROR: attempting to freeze an non-pretrained model!')
        
    # clear param store
    pyro.clear_param_store()
    # setup the VAE
    topic_model = ProdLDA(pwd.vocab_size, NUM_TOPICS, NUM_PROTOTYPES, HIDDEN_DIM, DROPOUT_RATE, DEVICE, frozen=frozen).to(DEVICE)
    
    if pretrained:
        saved_model_dict = torch.load("../data/pretrained_models/pretrained_20ng_noOF.pt", map_location=DEVICE)
        topic_model.load_state_dict(saved_model_dict['model'], strict=False)
        # svi_guide = saved_model_dict['guide']
        # pyro.get_param_store().load("../data/pretrained_models/pretrained_params_20ng_2000vocab.pt")
    
    if frozen:
        for param in topic_model.encoder.parameters():
            param.requires_grad=False
        for param in topic_model.decoder.parameters():
            param.requires_grad=False
        topic_model.frozen = True
        
    # setup the optimizer
    optimizer = ClippedAdam(adam_args)
    # setup the inference algorithm
    svi = SVI(topic_model.model, topic_model.guide, optimizer, loss=custom_elbo)
    

    
    return topic_model, svi

In [11]:
def trace_model(topic_model, dl_train):
    batch_1 = next(iter(dl_train))
    dat_a = batch_1['a'].to(DEVICE).squeeze()
    dat_b = batch_1['b'].to(DEVICE).squeeze()
    dat_label = batch_1['label'].to(DEVICE).squeeze().type(torch.float32)
    dat_observed = batch_1['observed'].to(DEVICE).squeeze().type(torch.bool)

In [12]:
def get_map(topic_model):
    x = torch.tensor(pwd.bows_val.toarray()).to(DEVICE).type(torch.float)
    with torch.no_grad():
        z_loc, z_scale = topic_model.encoder(x)

    features = z_loc.cpu()
    features = torch.softmax(features, axis=-1)
    features = features.numpy()

    trans = umap.UMAP(n_neighbors=15, random_state=42, min_dist=0.1).fit(features)
    return trans

## Training

In [ ]:
logging = True
pretrained = True
frozen = False
run_id = "s23_noOF_logtheta10_simscale10_5prototypes_lr0003"

run_name_original = ("pretrained_" if pretrained else "unpretrained_") + ("frozen_" if frozen else "unfrozen_") + "20ng_" + run_id

train_elbo = []
test_elbo = []
test_celoss = []
test_klloss = []

print("Beginning Training")

# training loop

for p_percentage in [.01]:
    pyro.set_rng_seed(0)
    run_name = run_name_original + f"_lp{int(p_percentage * 100)}"
    print(f"Training for percentage: {p_percentage}")
    best_accuracy = 0
    pwd, dl_train, dl_val, dl_test = get_data(p_percentage)
    topic_model, svi = init_model(pretrained=pretrained, frozen=frozen, pwd=pwd)
    if logging:
        run = wandb.init(project="prodLDA_pairs", entity="witw", )
        wandb.run.name = run_name 
        wandb.watch(topic_model, log='all', log_freq=100)
    trace_model(topic_model, dl_train=dl_train)
    for epoch in range(NUM_EPOCHS):
        trans = get_map(topic_model)
        total_epoch_loss_train, _, _ = train(svi, topic_model, dl_train, device=DEVICE, print_debug=True, epoch=epoch, latest_plotted_p=latest_plotted_p, trans = trans)
        train_elbo.append(-total_epoch_loss_train)

        if logging:
            wandb.log({'epoch': epoch,
                       'train_elbo': total_epoch_loss_train})

        print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))

        if epoch % TEST_FREQUENCY == 0:
            # report test diagnostics
            total_epoch_loss_test, total_epoch_celoss_test, total_epoch_klloss_test, test_acc, test_auroc = evaluate(svi, topic_model, dl_val, device=DEVICE)
            test_elbo.append(-total_epoch_loss_test)
            test_celoss.append(total_epoch_celoss_test)
            test_klloss.append(total_epoch_klloss_test)
    #         print(x)
            if logging:
                wandb.log({'epoch': epoch,
                           'test_acc': test_acc,
                           'test_auroc': test_auroc,
                           'test_elbo': total_epoch_loss_test,
                           'test_entropy': total_epoch_celoss_test,
                           'test_kl': total_epoch_klloss_test})

            print("\nEvaluation: ")
            print("[epoch %03d]  average elbo loss: %.4f" % (epoch, total_epoch_loss_test))
            print("              average ce loss:   %.4f" % (total_epoch_celoss_test))
            print("              average kld loss:  %.4f" % (total_epoch_klloss_test))
            print("              average accuracy:     %.4f" % (test_acc))
            print("              average auroc:  %.4f\n" % (test_auroc))
            if test_acc > best_accuracy:
                best_accuracy = test_acc
                torch.save({"model" : topic_model.state_dict()}, f"../data/20ng_checkpoints/labeled_percentage_sweep/bestmodel_" + run_name + ".pt")
    torch.save({"model" : topic_model.state_dict()},  f"../data/20ng_checkpoints/labeled_percentage_sweep/lastmodel_" + run_name + ".pt")


Beginning Training
Training for percentage: 0.01


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dsaad (witw). Use `wandb login --relogin` to force relogin


[epoch 000]  average training loss: 495.9047

Evaluation: 
[epoch 000]  average elbo loss: 484.9500
              average ce loss:   14.6285
              average kld loss:  53.7094
              average accuracy:     0.5472
              average auroc:  0.6091

[epoch 001]  average training loss: 485.3542

Evaluation: 
[epoch 001]  average elbo loss: 480.3098
              average ce loss:   14.6219
              average kld loss:  55.0881
              average accuracy:     0.5518
              average auroc:  0.6143

[epoch 002]  average training loss: 479.6507

Evaluation: 
[epoch 002]  average elbo loss: 482.9033
              average ce loss:   14.5618
              average kld loss:  63.8692
              average accuracy:     0.5637
              average auroc:  0.6213

[epoch 003]  average training loss: 473.8896

Evaluation: 
[epoch 003]  average elbo loss: 464.3277
              average ce loss:   14.4605
              average kld loss:  57.6087
              average accurac

In [ ]:
tok2ix = pwd.vectorizer.vocabulary_
ix2tok = {v:k for k,v in tok2ix.items()}
with torch.no_grad():
    phi_decoded = topic_model.decoder(phi.to(DEVICE))
top_term_ixs = phi_decoded.argsort(axis=-1, descending=True)[:,:15]

topic_terms = []
for jx, topic in enumerate(top_term_ixs):
    # print(f"THEME {jx+1}")
    terms = [ix2tok[ix.item()] for ix in topic]
    # for term in terms:
        # print(term)
    topic_terms.append(terms)
    # print()
    
df = pd.DataFrame(topic_terms)
df = df.transpose()
df.columns = ["Theme A", "Theme B", "Theme C", "Theme D", "Theme E", "Theme F", "Theme G"]
df.style.hide_index()

### 